# IK Velocity Unconstrained

In [1]:
import pinocchio as pin
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtCore import Qt, QTimer
from quadprog import solve_qp  # Install with `pip install quadprog`
import os

# Load the robot model
urdf_path = os.path.join("model.urdf")  # Replace with the path to your URDF file
model = pin.buildModelFromUrdf(urdf_path)
data = model.createData()

# Visualization setup
visual_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.VISUAL)
collision_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.COLLISION)
viz = pin.visualize.MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(loadModel=True)

# End-effector frame
end_effector_frame = model.getFrameId("lbr_iiwa_link_7")  # Replace with your end-effector frame name

# Initialize joint configuration
q = pin.neutral(model)
viz.display(q)

# Velocity scaling
velocity_scale = 0.1  # Adjust this for desired velocity magnitude
dt = 0.05  # Time step for integration
damping = 1e-6  # Regularization factor

# Key-to-twist mapping
key_twist_mapping = {
    Qt.Key_W: np.array([velocity_scale, 0, 0, 0, 0, 0]),  # Forward
    Qt.Key_S: np.array([-velocity_scale, 0, 0, 0, 0, 0]), # Backward
    Qt.Key_A: np.array([0, velocity_scale, 0, 0, 0, 0]), # Left
    Qt.Key_D: np.array([0, -velocity_scale, 0, 0, 0, 0]), # Right
    Qt.Key_Q: np.array([0, 0, velocity_scale, 0, 0, 0]), # Up
    Qt.Key_E: np.array([0, 0, -velocity_scale, 0, 0, 0]), # Down
    Qt.Key_J: np.array([0, 0, 0, -velocity_scale, 0, 0]), # Rotate around x towards left
    Qt.Key_L: np.array([0, 0, 0, velocity_scale, 0, 0]), # Rotate around x towards right
    Qt.Key_I: np.array([0, 0, 0, 0, velocity_scale, 0]), # Rotate around y down
    Qt.Key_K: np.array([0, 0, 0, 0, -velocity_scale, 0]), # Rotate around y up
    Qt.Key_U: np.array([0, 0, 0, 0, 0, velocity_scale]), # Yaw left
    Qt.Key_O: np.array([0, 0, 0, 0, 0, -velocity_scale]), # Yaw right
}

class VelocityIKController(QWidget):
    def __init__(self):
        super().__init__()
        self.pressed_keys = set()
        self.timer = QTimer()
        self.timer.timeout.connect(self.control_loop)
        self.timer.start(int(dt * 1000))

    def keyPressEvent(self, event):
        self.pressed_keys.add(event.key())

    def keyReleaseEvent(self, event):
        self.pressed_keys.discard(event.key())

    def compute_desired_twist(self):
        desired_twist = np.zeros(6)
        for key in self.pressed_keys:
            if key in key_twist_mapping:
                desired_twist += key_twist_mapping[key]
        return desired_twist

    def control_loop(self):
        global q

        # Compute forward kinematics and Jacobian
        pin.forwardKinematics(model, data, q)
        pin.updateFramePlacements(model, data)
        J = pin.computeFrameJacobian(model, data, q, end_effector_frame, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)

        # Get the desired twist from key input
        desired_twist = self.compute_desired_twist()

        if np.linalg.norm(desired_twist) > 1e-6:  # If there is a desired motion
            # Quadratic program matrices
            H = J.T @ J + damping * np.eye(model.nv)  # Regularized Hessian
            g = -J.T @ desired_twist  # Gradient term

            # Solve the quadratic program
            theta_dot = solve_qp(H, g)[0]

            # Update joint configuration using integration
            q = pin.integrate(model, q, theta_dot * dt)
            viz.display(q)

if __name__ == "__main__":
    app = QApplication([])
    controller = VelocityIKController()
    controller.show()
    app.exec_()


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


#  Constrained Velocity Kinematics

In [ ]:
import pinocchio as pin
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtCore import Qt, QTimer
from quadprog import solve_qp  # Install with `pip install quadprog`
import os

# Load the robot model
urdf_path = os.path.join("model.urdf")  # Replace with the path to your URDF file
model = pin.buildModelFromUrdf(urdf_path)
data = model.createData()

# Visualization setup
visual_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.VISUAL)
collision_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.COLLISION)
viz = pin.visualize.MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(loadModel=True)

# End-effector frame
end_effector_frame = model.getFrameId("lbr_iiwa_link_7")  # Replace with your end-effector frame name

# Initialize joint configuration
q = pin.neutral(model)
viz.display(q)

# Velocity scaling
velocity_scale = 0.1  # Adjust this for desired velocity magnitude
dt = 0.05  # Time step for integration
damping = 1e-6  # Regularization factor

# Joint limits
q_min = model.lowerPositionLimit
q_max = model.upperPositionLimit

# Key-to-twist mapping
key_twist_mapping = {
    Qt.Key_W: np.array([velocity_scale, 0, 0, 0, 0, 0]),  # Forward
    Qt.Key_S: np.array([-velocity_scale, 0, 0, 0, 0, 0]), # Backward
    Qt.Key_A: np.array([0, velocity_scale, 0, 0, 0, 0]), # Left
    Qt.Key_D: np.array([0, -velocity_scale, 0, 0, 0, 0]), # Right
    Qt.Key_Q: np.array([0, 0, velocity_scale, 0, 0, 0]), # Up
    Qt.Key_E: np.array([0, 0, -velocity_scale, 0, 0, 0]), # Down
    Qt.Key_J: np.array([0, 0, 0, -velocity_scale, 0, 0]), # Rotate around x towards left
    Qt.Key_L: np.array([0, 0, 0, velocity_scale, 0, 0]), # Rotate around x towards right
    Qt.Key_I: np.array([0, 0, 0, 0, velocity_scale, 0]), # Rotate around y down
    Qt.Key_K: np.array([0, 0, 0, 0, -velocity_scale, 0]), # Rotate around y up
    Qt.Key_U: np.array([0, 0, 0, 0, 0, velocity_scale]), # Yaw left
    Qt.Key_O: np.array([0, 0, 0, 0, 0, -velocity_scale]), # Yaw right
}

class VelocityIKController(QWidget):
    def __init__(self):
        super().__init__()
        self.pressed_keys = set()
        self.timer = QTimer()
        self.timer.timeout.connect(self.control_loop)
        self.timer.start(int(dt * 1000))

    def keyPressEvent(self, event):
        self.pressed_keys.add(event.key())

    def keyReleaseEvent(self, event):
        self.pressed_keys.discard(event.key())

    def compute_desired_twist(self):
        desired_twist = np.zeros(6)
        for key in self.pressed_keys:
            if key in key_twist_mapping:
                desired_twist += key_twist_mapping[key]
        return desired_twist

    def control_loop(self):
        global q

        # Compute forward kinematics and Jacobian
        pin.forwardKinematics(model, data, q)
        pin.updateFramePlacements(model, data)
        pin.computeJointJacobians(model, data, q)

        J = pin.computeFrameJacobian(model, data, q, end_effector_frame, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)

        # Get the desired twist from key input
        desired_twist = self.compute_desired_twist()

        if np.linalg.norm(desired_twist) > 1e-6:  # If there is a desired motion
            # Quadratic program matrices
            H = J.T @ J + damping * np.eye(model.nv)  # Regularized Hessian
            g = -J.T @ desired_twist  # Gradient term

            # Inequality constraints for joint limits
            theta_dot_max = 1.0 * np.ones(model.nv)
            theta_dot_min = -1.0 * np.ones(model.nv)

            C = np.vstack([np.eye(model.nv), -np.eye(model.nv)])
            b = np.hstack([theta_dot_min, -theta_dot_max])

            # Solve the quadratic program
            theta_dot = solve_qp(H, g, C.T, b)[0]

            # Update joint configuration using integration
            q = pin.integrate(model, q, theta_dot * dt)
            viz.display(q)

if __name__ == "__main__":
    app = QApplication([])
    controller = VelocityIKController()
    controller.show()
    app.exec_()


# Velocity Newton Raphson

In [1]:
import pinocchio as pin
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtCore import Qt, QTimer
import os

# Load the robot model
urdf_path = os.path.join("model.urdf")  # Replace with the path to your URDF file
model = pin.buildModelFromUrdf(urdf_path)
data = model.createData()

# Visualization setup
visual_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.VISUAL)
collision_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.COLLISION)
viz = pin.visualize.MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(loadModel=True)

# End-effector frame
end_effector_frame = model.getFrameId("lbr_iiwa_link_7")  # Replace with your end-effector frame name

# Initialize joint configuration
q = pin.neutral(model)
viz.display(q)

# Velocity scaling
velocity_scale = 0.1  # Adjust this for desired velocity magnitude
dt = 0.05  # Time step for integration

# Key-to-twist mapping
key_twist_mapping = {
    Qt.Key_W: np.array([velocity_scale, 0, 0, 0, 0, 0]),  # Forward
    Qt.Key_S: np.array([-velocity_scale, 0, 0, 0, 0, 0]), # Backward
    Qt.Key_A: np.array([0, velocity_scale, 0, 0, 0, 0]), # Left
    Qt.Key_D: np.array([0, -velocity_scale, 0, 0, 0, 0]), # Right
    Qt.Key_Q: np.array([0, 0, velocity_scale, 0, 0, 0]), # Up
    Qt.Key_E: np.array([0, 0, -velocity_scale, 0, 0, 0]), # Down
    Qt.Key_J: np.array([0, 0, 0, -velocity_scale, 0, 0]), # Rotate around x towards left
    Qt.Key_L: np.array([0, 0, 0, velocity_scale, 0, 0]), # Rotate around x towards right
    Qt.Key_I: np.array([0, 0, 0, 0, velocity_scale, 0]), # Rotate around y down
    Qt.Key_K: np.array([0, 0, 0, 0, -velocity_scale, 0]), # Rotate around y up
    Qt.Key_U: np.array([0, 0, 0, 0, 0, velocity_scale]), # Yaw left
    Qt.Key_O: np.array([0, 0, 0, 0, 0, -velocity_scale]), # Yaw right
}

class VelocityIKController(QWidget):
    def __init__(self):
        super().__init__()
        self.pressed_keys = set()
        self.timer = QTimer()
        self.timer.timeout.connect(self.control_loop)
        self.timer.start(int(dt * 1000))

    def keyPressEvent(self, event):
        self.pressed_keys.add(event.key())

    def keyReleaseEvent(self, event):
        self.pressed_keys.discard(event.key())

    def compute_desired_twist(self):
        desired_twist = np.zeros(6)
        for key in self.pressed_keys:
            if key in key_twist_mapping:
                desired_twist += key_twist_mapping[key]
        return desired_twist

    def control_loop(self):
        global q

        # Compute forward kinematics and Jacobian
        pin.forwardKinematics(model, data, q)
        pin.updateFramePlacements(model, data)
        J = pin.computeFrameJacobian(model, data, q, end_effector_frame, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)

        # Get the desired twist from key input
        desired_twist = self.compute_desired_twist()

        if np.linalg.norm(desired_twist) > 1e-6:  # If there is a desired motion
            # Solve for joint velocities using least squares
            dq = np.linalg.pinv(J) @ desired_twist

            # Update joint configuration using integration
            q = pin.integrate(model, q, dq * dt)
            viz.display(q)

if __name__ == "__main__":
    app = QApplication([])
    controller = VelocityIKController()
    controller.show()
    app.exec_()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


# Collision check Velocity Kinematics

In [1]:
import pinocchio as pin
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtCore import Qt, QTimer
from quadprog import solve_qp  # Install with `pip install quadprog`
import os

# Load the robot model
urdf_path = os.path.join("model.urdf")  # Replace with the path to your URDF file
model = pin.buildModelFromUrdf(urdf_path)
data = model.createData()

# Collision setup
collision_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.COLLISION)
collision_data = pin.GeometryData(collision_model)

# Visualization setup
visual_model = pin.buildGeomFromUrdf(model, urdf_path, pin.GeometryType.VISUAL)
viz = pin.visualize.MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(loadModel=True)

# End-effector frame
end_effector_frame = model.getFrameId("lbr_iiwa_link_7")  # Replace with your end-effector frame name

# Initialize joint configuration
q = pin.neutral(model)
viz.display(q)

# Velocity scaling
velocity_scale = 0.1  # Adjust this for desired velocity magnitude
dt = 0.05  # Time step for integration
damping = 1e-6  # Regularization factor

# Joint limits
q_min = model.lowerPositionLimit
q_max = model.upperPositionLimit

# Key-to-twist mapping
key_twist_mapping = {
    Qt.Key_W: np.array([velocity_scale, 0, 0, 0, 0, 0]),  # Forward
    Qt.Key_S: np.array([-velocity_scale, 0, 0, 0, 0, 0]), # Backward
    Qt.Key_A: np.array([0, velocity_scale, 0, 0, 0, 0]), # Left
    Qt.Key_D: np.array([0, -velocity_scale, 0, 0, 0, 0]), # Right
    Qt.Key_Q: np.array([0, 0, velocity_scale, 0, 0, 0]), # Up
    Qt.Key_E: np.array([0, 0, -velocity_scale, 0, 0, 0]), # Down
    Qt.Key_J: np.array([0, 0, 0, -velocity_scale, 0, 0]), # Rotate around x towards left
    Qt.Key_L: np.array([0, 0, 0, velocity_scale, 0, 0]), # Rotate around x towards right
    Qt.Key_I: np.array([0, 0, 0, 0, velocity_scale, 0]), # Rotate around y down
    Qt.Key_K: np.array([0, 0, 0, 0, -velocity_scale, 0]), # Rotate around y up
    Qt.Key_U: np.array([0, 0, 0, 0, 0, velocity_scale]), # Yaw left
    Qt.Key_O: np.array([0, 0, 0, 0, 0, -velocity_scale]), # Yaw right
}

class VelocityIKController(QWidget):
    def __init__(self):
        super().__init__()
        self.pressed_keys = set()
        self.timer = QTimer()
        self.timer.timeout.connect(self.control_loop)
        self.timer.start(int(dt * 1000))

    def keyPressEvent(self, event):
        self.pressed_keys.add(event.key())

    def keyReleaseEvent(self, event):
        self.pressed_keys.discard(event.key())

    def compute_desired_twist(self):
        desired_twist = np.zeros(6)
        for key in self.pressed_keys:
            if key in key_twist_mapping:
                desired_twist += key_twist_mapping[key]
        return desired_twist

    def check_collisions(self, q_next):
        """
        Perform comprehensive collision checking with more robust detection.
        
        Args:
            q_next (np.ndarray): Next joint configuration to check
            collision_threshold (float): Minimum distance to consider as a collision
        
        Returns:
            bool: True if collision detected, False otherwise
        """
        # Create a fresh GeometryData for thorough collision checking
        collision_data = pin.GeometryData(collision_model)
        
        # Update geometry placements for the new configuration
        pin.updateGeometryPlacements(model, data, collision_model, collision_data, q_next)
        
        # Check for collisions between all pairs
        try:
            # Compute distances between all collision pairs
            if pin.computeCollisions(collision_model, collision_data):
                return True
            
            return False
        
        except Exception as e:
            print(f"Collision detection error: {e}")
            # If there's an error, it's safer to prevent movement
            return True

    def control_loop(self):
        global q

        # Compute forward kinematics and Jacobian
        pin.forwardKinematics(model, data, q)
        pin.updateFramePlacements(model, data)
        pin.computeJointJacobians(model, data, q)

        J = pin.computeFrameJacobian(model, data, q, end_effector_frame, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)

        # Get the desired twist from key input
        desired_twist = self.compute_desired_twist()

        if np.linalg.norm(desired_twist) > 1e-6:  # If there is a desired motion
            # Quadratic program matrices
            H = J.T @ J + damping * np.eye(model.nv)  # Regularized Hessian
            g = -J.T @ desired_twist  # Gradient term

            # Inequality constraints for joint limits
            theta_dot_max = 1.0 * np.ones(model.nv)
            theta_dot_min = -1.0 * np.ones(model.nv)

            C = np.vstack([np.eye(model.nv), -np.eye(model.nv)])
            b = np.hstack([theta_dot_min, -theta_dot_max])

            # Solve the quadratic program
            theta_dot = solve_qp(H, g, C.T, b)[0]

            # Simulate next configuration
            q_next = pin.integrate(model, q, theta_dot * dt)

            # Check for collisions
            if not self.check_collisions(q_next):  # Only update if no collisions
                q = q_next
                viz.display(q)

if __name__ == "__main__":
    app = QApplication([])
    controller = VelocityIKController()
    controller.show()
    app.exec_()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Collision detection error: Python argument types in
    pinocchio.pinocchio_pywrap.computeCollisions(GeometryModel, GeometryData)
did not match C++ signature:
    computeCollisions(pinocchio::ModelTpl<double, 0, pinocchio::JointCollectionDefaultTpl> model, pinocchio::DataTpl<double, 0, pinocchio::JointCollectionDefaultTpl> {lvalue} data, pinocchio::GeometryModel geometry_model, pinocchio::GeometryData {lvalue} geometry_data, Eigen::MatrixBase<Eigen::Matrix<double, -1, 1, 0, -1, 1> > q, bool stop_at_first_collision)
    computeCollisions(pinocchio::GeometryModel geometry_model, pinocchio::GeometryData {lvalue} geometry_data, bool stop_at_first_collision)
Collision detection error: Python argument types in
    pinocchio.pinocchio_pywrap.computeCollisions(GeometryModel, GeometryData)
did not match C++ signature:
    computeCollisions(pinocchio::ModelTpl<double, 0, pinocchio::JointCollectionDefaultTpl>